In [ ]:
import requests
from bs4 import BeautifulSoup

site = 'http://www.cdiscount.com/search/10/ordinateur+<brand>.html'

Params = {
    'page': 1
}

# Affiche la page de résultats de la recherche pour la marque et le numéro de page spécifiés;
# retourne le contenu de la page trouvée
def recherche_page(brand, page):
    url = site.replace('<brand>', brand)
    Params['page'] = page
    return requests.get(url, Params)


# Récupère la liste des prix de vente au détail recommandés et des prix réduits pour les ordinateurs de la marque.

def liste(brand, discounts_only=False, page=1):
    data = recherche_page(brand, page)
    results = []
    parser = BeautifulSoup(data.text, 'html.parser')
    for node in parser.find_all(class_="prdtBZPrice"):
        price_node = node.find(class_="prdtPrice")
        if price_node:
            price_node = price_node.find(class_="price")
            try:
                discounted_price = extract_price(price_node)
                discount_node = node.find(class_="prdtPrSt")
                recommended_price = discounted_price
                try:
                    if discount_node:
                        recommended_price = extract_price(discount_node)
                    elif discounts_only:
                        continue
                except ValueError:
                    pass
                results.append([discounted_price, recommended_price])
            except ValueError:
                pass
    return results

# Extrait les prix
def extract_price(node):
    return float(node.text.replace(u'\xa0', '').replace(u' ', '').replace(u'€', '.').replace(u',', u'.'))

# Récupère la liste des prix recommandés et des prix réduits pour les ordinateurs de la marque spécifiée.
def cdiscount(brand, discounts_only=False, pages=10):
    results = []
    for page in range(1, pages + 1):
        results.extend(liste(brand, discounts_only=discounts_only, page=page))
    return results

def test(pages):
    reduct_dell = cdiscount('dell', discounts_only=True, pages=pages)
    reduct_acer = cdiscount('acer', discounts_only=True, pages=pages)
    moyenne_discounts_dell = 1 - sum(map(lambda v: v[0], discounts_dell)) / sum(map(lambda v: v[1], reduct_dell))
    moyenne_discounts_acer = 1 - (sum(map(lambda v: v[0], discounts_acer)) / sum(map(lambda v: v[1], reduct_acer)))
    print("Average discounts for Dell from the %d first pages: %f" % (pages, moyenne_discounts_dell))
    print("Average discounts for Acer from the %d first pages: %f" % (pages, moyenne_discounts_acer))

test(3)

